In [1]:
# Code to enable this notebook to import from libraries
import os
import sys
module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sns
import csv
from scripts.mockUtilities import *
from scripts.utilities import *

In [3]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pywt
import pywt.data
from sklearn.cluster import SpectralClustering
from matplotlib.colors import ListedColormap
from scipy.sparse import csr_matrix
from scipy.io import mmread
from sklearn.mixture import GaussianMixture
%matplotlib inline

In [4]:
# Set seed for reproducibility
np.random.seed(999)

In [5]:
from matplotlib.colors import ListedColormap

## Loading Data

In [6]:
counts = mmread(r'..\..\data\10x_visium_LIBD\sample9\counts_hvg.mtx')
counts_sct = mmread(r'..\..\data\10x_visium_LIBD\sample9\sct_transformed_counts.mtx')
coords = pd.read_csv(r'..\..\data\10x_visium_LIBD\sample9\coords.csv')
meta_data = pd.read_csv(r'..\..\data\10x_visium_LIBD\sample9\meta.csv')

In [7]:
coords = coords.rename(columns={'x_coord': 'x', 'y_coord': 'y'})

# counts data
cell_feature_ori = counts
cell_feature_ori = (cell_feature_ori.toarray())
meta_data["feature_ori"] =  [cell_feature_ori[:,x] for x in range(meta_data.shape[0])] 

# sctransformed counts data
cell_feature_sct = counts_sct
cell_feature_sct = (cell_feature_sct.toarray())
meta_data["feature_sct"] =  [cell_feature_sct[:,x] for x in range(meta_data.shape[0])] 

meta_data = meta_data[['barcode', 'feature_sct', 'feature_ori', 'spatialLIBD']]
rna = pd.concat([coords, meta_data], axis=1)
rna['original index'] = rna.index
rna = rna.dropna()
rna['spatialLIBDCode'] = rna['spatialLIBD'].astype('category').cat.codes
rna = rna.reset_index(drop=True)

## Processing data

We will be using the feature_sct data (sctransformed data)

In [8]:
# Set the settings for the data
N_GENES = len(rna["feature_sct"][0])

# Set the settings for the gridding of the data
MIN_X = 130
MAX_X = 500
MIN_Y = -510
MAX_Y = -100
N_X_INDICES = 2**6
N_Y_INDICES = N_X_INDICES
N_CELLS = N_X_INDICES * N_Y_INDICES

# Calculate the size of each cell in the grid (these are boxes in the thesis)
cell_size_x = (MAX_X - MIN_X) / N_X_INDICES
cell_size_y = (MAX_Y - MIN_Y) / N_Y_INDICES

# Assign each (x, y) point to a grid cell
rna['grid_x'] = ((rna['x'] - MIN_X) // cell_size_x).astype(int)
rna['grid_y'] = ((rna['y'] - MIN_Y) // cell_size_y).astype(int)

# Group by grid cells (grid_x, grid_y) and calculate the average feature vector for each cell
grouped = rna.groupby(['grid_y', 'grid_x'])['feature_sct'].apply(lambda x: np.mean(np.vstack(x), axis=0))
fill_feature_vector = np.zeros(N_GENES)
full_index = pd.MultiIndex.from_product([range(N_Y_INDICES), range(N_X_INDICES)], names=['y_index', 'x_index'])
grouped_reindexed = grouped.reindex(full_index, fill_value=fill_feature_vector)
grid_rna = pd.DataFrame(grouped_reindexed.tolist(), index=grouped_reindexed.index)

# Rename the columns to gene_indicator_n where n is the element index of the feature vector
grid_rna.columns = [f'gene_indicator_{i}' for i in range(grid_rna.shape[1])]

# Reset index to have y_index and x_index as named columns
grid_rna.index.names = ['y_index', 'x_index']

In [9]:
result_rna_unnormalised = grid_rna.copy()
result_rna_unnormalised['feature'] = result_rna_unnormalised.apply(lambda row: row.tolist(), axis=1)
result_rna_unnormalised = result_rna_unnormalised.drop(result_rna_unnormalised.columns[:-1], axis=1)
result_rna_unnormalised = result_rna_unnormalised.reset_index()

In [10]:
# Perform standardization 
from sklearn.preprocessing import StandardScaler
result_rna = result_rna_unnormalised.copy()
matrix = result_rna_unnormalised['feature'].tolist()
matrix = np.array(matrix)
scaler = StandardScaler()
matrix_standardized = scaler.fit_transform(matrix)
standardized_features = matrix_standardized.tolist()
result_rna['feature'] = standardized_features

In [11]:
assert np.allclose(matrix_standardized.mean(axis=0),np.array([0.0]*N_GENES))
assert np.allclose(matrix_standardized.std(axis=0),np.array([1.0]*N_GENES))

## Settings for inference

In [12]:
N_FACTORS = 7 # M in thesis
N_LENGTH_SCALES = 4 # D in thesis
n_resolutions = N_LENGTH_SCALES + 1

## Run PCA

In [13]:
n_components = N_FACTORS
feats = np.vstack(result_rna['feature'].to_numpy())
pca = PCA(n_components=n_components)
flattened_L_pca = pca.fit_transform(feats)
F_pca = pca.components_ # F matrix, factor vs feature

L_pca = [np.zeros((N_Y_INDICES, N_X_INDICES)) for _ in range(n_components)]
for i, row in result_rna.iterrows():
    y_idx = row['y_index']
    x_idx = row['x_index']
    
    for component_index in range(n_components):
        L_pca[component_index][y_idx, x_idx] = flattened_L_pca[i, component_index]

LF_pca = pca.inverse_transform(flattened_L_pca).transpose()

## Run WaviFM

In [14]:
from IPython.display import display, HTML
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from sklearn.mixture import GaussianMixture
from scripts.cavi_plot_utilities import *
from scripts.cavi import *
import copy
from scripts.cavi_utilities import *
from scripts.cavi_evaluation import *
from scripts.utilities import *
from scripts.run_wavifm import *

%matplotlib inline

In [15]:
n_factors = N_FACTORS
p_eta_shape = (n_factors,)
p_pi_shape = (n_resolutions,)

In [16]:
# Setup priors (mainly for imposing strong levels of sparsity for finer resolutions)
priors = {
    "log_p_pi": np.log(np.array((31/32,31/32,1/4,1/8,1/16))).astype(np.float64),
    "log_p_eta": np.log(np.full(p_eta_shape, 0.2).astype(np.float64)),
}
assert priors["log_p_pi"].shape == p_pi_shape

In [17]:
results = run_wavifm(
    result_rna=result_rna,
    n_length_scales=N_LENGTH_SCALES,
    n_factors=N_FACTORS,
    n_x_indices=N_X_INDICES,
    n_y_indices=N_Y_INDICES,
    max_iterations=1000,
    relative_elbo_threshold=0.00001,
    n_init=5,
    priors=priors
)
param_results = results["parameters"]

Initialisation 1:
	ELBO = -10462413.902255729
	#Iterations = 53
	Time taken (s) = 768.9727478027344
Initialisation 2:
	ELBO = -10476976.900444908
	#Iterations = 395
	Time taken (s) = 5674.423114299774
Initialisation 3:
	ELBO = -10493054.249996228
	#Iterations = 422
	Time taken (s) = 6106.735743522644
Initialisation 4:
	ELBO = -10489329.9289578
	#Iterations = 395
	Time taken (s) = 5645.811079025269
Initialisation 5:
	ELBO = -10456269.869452242
	#Iterations = 101
	Time taken (s) = 1494.4556975364685
Initialisation 5 has maximal ELBO and is returned


### Processing results

In [18]:
L_means = variational_approx_posterior_mean_L(param_results)
F_means = variational_approx_posterior_mean_F(param_results)
pi_means = variational_approx_posterior_mean_pi(param_results)
eta_means = variational_approx_posterior_mean_eta(param_results)

Get factor activities in spatial domain

In [19]:
def square_flattened_matrix(flattened_matrix):
    flattened_matrix_arr = np.array(flattened_matrix)
    n = int(np.sqrt(len(flattened_matrix_arr)))
    square_matrix = flattened_matrix_arr.reshape((n, n))
    return square_matrix

In [20]:
L_means_formatted = copy.deepcopy(L_means)
n_factors, n_features = np.array(param_results["mu_F"]).shape
mean_values = copy.deepcopy(param_results["mu_L"])

for l in range(n_factors):
    for i in range(len(param_results["mu_L"][l])):
        for j in range(len(param_results["mu_L"][l][i])):
            L_means_formatted[l][i][j] = square_flattened_matrix(L_means[l][i][j])

for l in range(n_factors):
    L_means_formatted[l][0] = square_flattened_matrix(L_means[l][0][0])

In [21]:
# Inverse Wavelet Transform on the factor loadings
idwt_L_means = [None]*n_factors

for l in range(n_factors):
    print(f"factor {l}")
    idwt_L_means[l] = pywt.waverec2(L_means_formatted[l], 'haar')

factor 0
factor 1
factor 2
factor 3
factor 4
factor 5
factor 6


### Export results

In [22]:
# Export inferred factor activities
np.savez('L_pca.npz', *L_pca)
np.savez('idwt_L_means.npz', *idwt_L_means)
np.savez('F_means.npz', *F_means)
np.savez('F_pca.npz', *F_pca)